# Project 01 : News Paper Scrapping

하나의 전자신문 회사에서 원하는 키워드에 해당되는 모든 뉴스를 수집 및 저장하는 코드를 작성하라

In [1]:
import re
import requests
from bs4 import BeautifulSoup
        
class newsScrapHK():
    def __init__(self,query):
        self.query = query
        self.curpage = '1'
        self.basic_url = "https://search.hankyung.com/apps.frm/search.news?query="
        self.url = self.basic_url + self.query + "&page=" + self.curpage
        newsScrapHK.make_soup(self)
        newsScrapHK.new_num(self)
        self.scrapBox = ""
        
    def make_soup(self):
        page = requests.get(self.url).text
        self.soup = BeautifulSoup(page,'html.parser')
        return self.soup
        
    def new_num(self):
        new_n = self.soup.find("h3", class_ = 'tit').text
        match = re.search('[\w\s\W]* (?P<num>[\s\d+\,]*[\d+])[건]',new_n)
        self.new_nums = int(match.group('num').replace(',',''))
        return self.new_nums
    
    def check_title(self):
        titles = self.soup.find_all('em',class_ = 'tit')
        for i, title in enumerate(titles):
            print('%d : ' % (i+1), title.text.strip())
            
    def page_link(self):
        sel_s = '#content > div.left_cont > div > div.section.hk_news > div > ul > li:nth-of-type('
        sel_e = ") > div > a"

        self.links = []
        try : 
            for i in range(1,11):
                line = self.soup.select(sel_s + str(i) + sel_e)
                self.links.append(line[0].get('href'))
                
        except IndexError:
            pass

        return self.links
    
    def news_scrap(self):
        self.scrapBox = ""
        for link in self.links:
            page1 = requests.get(link).text
            page_soup = BeautifulSoup(page1,'html.parser')
            try : 
                article = page_soup.find(id="articletxt").text.strip()
            except AttributeError:
                article = ""
            self.scrapBox += ''.join(re.sub('\n|   ',' ',article))
        return self.scrapBox
    
    def change_page_num(self,num):
        self.url = self.basic_url + self.query + "&page=" + str(num)
        newsScrapHK.make_soup(self)
        newsScrapHK.page_link(self)

    def change_query(self,query):
        self.query = query
        self.url = self.basic_url + self.query + "&page=" + self.curpage
        newsScrapHK.make_soup(self)
        newsScrapHK.page_link(self)
        newsScrapHK.new_num(self)

    def news_scrap_all(self):
        self.scrapBox = ""
        for i in range(1,self.new_nums // 10):
            newsScrapHK.change_page_num(self,i)
            newsScrapHK.page_link(self)
            newsScrapHK.news_scrap(self)
        return self.scrapBox
    
    def news2txt(self):
        
        with open('newsScraps.txt','w',-1,'utf-8') as f:
            f.write(self.scrapBox)
        print('Done')
            

In [2]:
hk = newsScrapHK('유가')

In [3]:
hk.new_num()

83979

In [4]:
hk.check_title()

1 :  '희대의 탈출극' 곤 닛산 前 회장, 경비업체 감시 중단된 틈 타 도주
2 :  "곤 前회장, 경비업체 고소 경고로 감시 중단시키고 도주"
3 :  이란 '호르무즈 봉쇄' 보복하나…국내 기름값 덩달아 불안
4 :  이란의 '가혹한 보복' 옵션은?…사이버전·원유봉쇄 등 거론
5 :  이란, 미국에 '최고 보복' 선언…중동위기 다시 최고조
6 :  美 이란 공습 중동 긴장으로 국제유가 '출렁'…국내 영향은
7 :  중동 리스크에 금융시장 긴장…금·엔화·원유 값 들썩
8 :  미-이란 위기에 뉴욕증시 3대지수 동반하락…유가·금값·채권↑
9 :  뉴욕증시, 중동 정세 일촉즉발…다우, 0.81% 하락 마감
10 :  美-이란 위기고조에 유가·금값 껑충…WTI 3.1%↑


In [5]:
hk.page_link()

['https://www.hankyung.com/international/article/2020010423317',
 'https://www.hankyung.com/society/article/202001042288Y',
 'https://www.hankyung.com/economy/article/2020010422587',
 'https://www.hankyung.com/society/article/202001042226Y',
 'https://www.hankyung.com/international/article/2020010421597',
 'https://www.hankyung.com/economy/article/202001042034Y',
 'https://www.hankyung.com/economy/article/202001042024Y',
 'https://www.hankyung.com/international/article/202001042023Y',
 'https://www.hankyung.com/finance/article/202001042012Y',
 'https://www.hankyung.com/international/article/202001041993Y']

In [6]:
hk.new_nums

83979

In [7]:
hk.change_query('파이썬')
hk.new_nums

87

In [8]:
hk.news_scrap_all()

'한양사이버대학교가 교육부 재정지원 특성화사업에 세 번 연속 선정됐다.3일 한양사이버대에 따르면 이 대학은 최근 교육부와 한국교육학술정보원이 주관하는 ‘2018년 성인학습자 역량 강화 교육콘텐츠 개발’사업을 수주했다. 2013년 선취업 후진학 지원사업, 2014년 NCS특성화 지원사업에 이어 세 번 연속으로 정부 재정지원사업에 선정됐다. 한양사이버대는 이날 한국교육학술정보원과 사업 계약과 관련한 협상을 마무리 짓고 본격적인 사업에 착수했다.‘성인 학습자 역량 강화 교육 콘텐츠 개발 사업’은 사이버대학이 생애주기별로 성인학습자에게 필요한 교육과정을 제공할 수 있도록 정부가 지원하는 사업이다. 올해는 전국 21개 사이버대학 중 8개 대학이 선정됐다. 대학별로 1억8000만원에서 2억1000만원 가량을 지원한다. 선정된 대학은 △19~24세 △25~40세 △41~65세 △65세 이후로 생애주기를 분류해 각 시기마다 성인 학습자의 역량 강화를 위한 맞춤형 단기 교육과정과 콘텐츠를 개발하고 운영하게 된다.한양사이버대는 이번 사업에서 총 7개의 콘텐츠로 구성된 ‘4C 인재를 육성하는 코딩강사 양성과정’을 제안했다. 코딩을 통해 전직을 희망하는 소프트웨어 개발자나 경력단절 여성, 취업준비생 뿐만 아니라 보수교육을 필요로 하는 중고등학교 코딩 교·강사, 자녀에게 코딩교육을 하고 싶어 하는 학부모까지 다양한 성인학습자를 위한 기초 코딩 교과목을 구성했다. 특히 코딩 교육에 대한 심리적 진입장벽을 낮추고 대중화를 위해 쉬운 내용으로 구성하되 보드게임, 스크래치, 파이썬, C 언어 등 여러 사용처와 난이도의 코딩 도구를 선정해 교육생이 선택적으로, 순차적으로 학습할 수 있도록 지원할 예정이다. 또 기존 코딩교육에서는 없는 ‘코딩실무영어’ 등의 과목을 개설해 학습자들이 프로그래밍 언어 학습시 도움을 받을 수 있도록 했다.구은서 기자 koo@hankyung.com“데이터분석가가 꿈이예요. 서울대 필독서를 어떤 순서로 읽으면 좋을까요” “고2 학생이예요. 저는 데이터분석가를 꿈꾸고 있

In [9]:
hk.news2txt()

Done
